In [83]:
import asyncio
import pandas as pd
from playwright.async_api import async_playwright
import json

async def main():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        try:
            await page.goto("https://www.sportsbookreview.com/betting-odds/nba-basketball/", wait_until="domcontentloaded")
            locator = page.locator("#__NEXT_DATA__")
            raw_json = await locator.inner_text()
            data = json.loads(raw_json)
        except Exception as e:
            print(f"Error: {e}")
        finally:
            await browser.close()
    return data

data = await main()

In [109]:
dframe = pd.DataFrame(columns=["away_TEAM", "home_TEAM","homeMoneyLine", "awayMoneyLine", "homeSpread","awaySpread"])
for index, x in enumerate((data['props']['pageProps']['oddsTables'][0]['oddsTableModel']['gameRows'])):
    homeMoneyLine, awayMoneyLine, homeSpread, awaySpread = 0,0,0,0
    for z in x["oddsViews"]:
        homeMoneyLine += z['currentLine']["homeOdds"]
        awayMoneyLine += z['currentLine']["awayOdds"]
        homeSpread += z['currentLine']['homeSpread']
        awaySpread += z['currentLine']['awaySpread']
    homeMoneyLine = round(homeMoneyLine/len(x['oddsViews']),2)
    awayMoneyLine = round(awayMoneyLine/len(x['oddsViews']),2)
    homeSpread = round(homeSpread/len(x['oddsViews']),2)
    awaySpread = round(awaySpread/len(x['oddsViews']),2)
    dframe.loc[index] = ({"away_TEAM":x['gameView']['awayTeam']['fullName'],
    "home_TEAM":x['gameView']['homeTeam']['fullName'],
    "home_Money_Line":homeMoneyLine,
    "away_Money_Line":awayMoneyLine,
    "homeSpread":homeSpread,
    "awaySpread":awaySpread
    }
    )
dframe

,away_TEAM,home_TEAM,homeMoneyLine,awayMoneyLine,homeSpread,awaySpread
0,New Orleans Pelicans,Brooklyn Nets,NaN,NaN,-3.58,3.58
1,Atlanta Hawks,Washington Wizards,NaN,NaN,9.42,-9.42
2,Golden State Warriors,Cleveland Cavaliers,NaN,NaN,-7.58,7.58
3,Milwaukee Bucks,Detroit Pistons,NaN,NaN,-12.58,12.58
4,LA Clippers,Minnesota Timberwolves,NaN,NaN,-9.17,9.17
5,Sacramento Kings,Miami Heat,NaN,NaN,-9.17,9.17
6,Houston Rockets,Dallas Mavericks,NaN,NaN,7.33,-7.33
